In [3]:
import pandas as pd
import os
import requests as rq

In [4]:
#한국관광 데이터랩 에서 서울특별시 인기관광지 데이터 가져오기

In [21]:
df = pd.read_csv(r'./data/20240928201108_인기관광지_전체.csv', encoding = 'cp949')

In [22]:
df

,순위,관광지명,주소,분류
0,1,김포국제공항국내선,서울 강서구 하늘길 111-0,교통시설
1,2,더현대서울,서울 영등포구 여의대로 108-0,백화점
2,3,서울역,서울 용산구 한강대로 405-0,교통시설
3,4,SRT수서역,서울 강남구 밤고개로 99-0,교통시설
4,5,코엑스,서울 강남구 영동대로 513-0,전시시설
...,...,...,...,...
95,96,롯데호텔월드,서울 송파구 올림픽로 240-0,호텔
96,97,잠실한강공원,서울 송파구 한가람로 65-0,도시공원
97,98,롯데백화점에비뉴엘월드타워점,서울 송파구 올림픽로 300-0,백화점
98,99,고척스카이돔,서울 구로구 경인로 430-0,레저스포츠시설


In [26]:
#관광지 분류만 모으기
tourlist_type = df['분류'].drop_duplicates()
tourlist_type_list = list(tourlist_type)

In [27]:
print(tourlist_type) 
#교통시설, 호텔은 주로 주차장이 확보되어 있으므로 제거(또한 관광지라고 보기 애매함)

0        교통시설
1         백화점
4        전시시설
6          시장
7         쇼핑몰
19       도시공원
20       테마공원
23         호텔
26       공연시설
53    레저스포츠시설
55      역사유적지
57     랜드마크관광
78       대형마트
Name: 분류, dtype: object


In [28]:
df = df[~df['분류'].isin(['교통시설', '호텔'])]

In [34]:
df = df[['관광지명', '주소', '분류']]
df

,관광지명,주소,분류
1,더현대서울,서울 영등포구 여의대로 108-0,백화점
4,코엑스,서울 강남구 영동대로 513-0,전시시설
5,신세계백화점강남점,서울 서초구 신반포로 176-0,백화점
6,가락동농수산물도매시장,서울 송파구 양재대로 932-0,시장
7,타임스퀘어,서울 영등포구 영중로 15-0,쇼핑몰
...,...,...,...
93,북서울꿈의숲,서울 강북구 월계로 173-0,도시공원
96,잠실한강공원,서울 송파구 한가람로 65-0,도시공원
97,롯데백화점에비뉴엘월드타워점,서울 송파구 올림픽로 300-0,백화점
98,고척스카이돔,서울 구로구 경인로 430-0,레저스포츠시설


In [35]:
df.count()

관광지명    72
주소      72
분류      72
dtype: int64

In [36]:
#주소를 기준으로 위도, 경도 구하기 (google map 활용)

In [ ]:
google_api_key = '' #google_api_key
# url = 'https://maps.googleapis.com/maps/api/geocode/outputFormat?parameters' 

In [37]:
def get_latitude_longitude(address): # 주소를 가지고 위도, 경도를 알아내기
    url = f'https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={google_api_key}'
    response = response = rq.get(url)
    if response.status_code == 200:
        results = response.json()
        if results['status'] == 'OK': #results status가 OK인지 확인
            latitude = results['results'][0]['geometry']['location']['lat']
            longitude = results['results'][0]['geometry']['location']['lng']
            return latitude, longitude
    return None, None

In [38]:
#tourlist 데이터프레임에 위도, 경도가 없으므로, 함수 적용
df[['위도', '경도']] = df['주소'].apply(lambda x: pd.Series(get_latitude_longitude(x)))
df

,관광지명,주소,분류,위도,경도
1,더현대서울,서울 영등포구 여의대로 108-0,백화점,37.526050,126.928296
4,코엑스,서울 강남구 영동대로 513-0,전시시설,37.512521,127.058819
5,신세계백화점강남점,서울 서초구 신반포로 176-0,백화점,37.504364,127.003621
6,가락동농수산물도매시장,서울 송파구 양재대로 932-0,시장,37.493482,127.111504
7,타임스퀘어,서울 영등포구 영중로 15-0,쇼핑몰,37.517075,126.903341
...,...,...,...,...,...
93,북서울꿈의숲,서울 강북구 월계로 173-0,도시공원,37.621771,127.041174
96,잠실한강공원,서울 송파구 한가람로 65-0,도시공원,37.518507,127.083144
97,롯데백화점에비뉴엘월드타워점,서울 송파구 올림픽로 300-0,백화점,37.513101,127.103433
98,고척스카이돔,서울 구로구 경인로 430-0,레저스포츠시설,37.498931,126.867087


In [42]:
#dataframe을 csv파일로 저장
os.getcwd()
df.to_csv('tourlist.csv', index = False, encoding = 'cp949')